# Analysis of Mushroom Dataset

**by Stefano Zamboni**

In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')



# Index

* [1. Introduction](#Introduction)
* [2. Dataset Exploration](#Dataset-Exploration)
    * [2.1 Overview](#Overview)
    * [2.2 Data visualization](#Visualization)
    * [2.3 Feature statistics](#Feature-Analysis)
* [3. Principal Component Analysis (PCA)](#Principal-Component-Analysis)
* [4. Comparison on classification over components](#Comparison-for-classification-of-differently-preprocessed-datasets)
    * [4.1 Datasets Evaluation](#Evaluation)
    * [4.2 Comparison](#Comparison)
* [5. Classification](#Classification)
    * [5.0 Regression vs Classification](#Regression-vs-Classification)
    * [5.1 Logistic Regression](#Logistic-Regression)
    * [5.2 Linear Discriminant Analysis (LDA)](#Linear-Discriminant-Analysis-[LDA])
    * [5.3 K-Nearest Neighbors](#K-Nearest-Neighbors)
    * [5.4 Decision Tree](#Decision-Tree)
    * [5.5 Random Forest](#Random-Forest)
    * [5.6 Support-Vector Machine](#Support-Vector-Machine)
    * [5.7 Results](#Results)
* [6. Conclusions](#Conclusions)


# Introduction

In this work, the goal is to analyze a dataset containing data about mushrooms, trying to discover correlation between features and model some classifiers to predict whether a mushroom is poisonous or edible. The full dataset can be viewed and downloaded [here](https://www.kaggle.com/uciml/mushroom-classification/version/1)


Each row of the dataset contains data relative to a mushroom observed, and it has several attributes (23). Here they are presented and briefly described:
<br>
* class: Edible (e) or Poisonous (p), the main class on which we will do predictions
* cap-shape: bell=b,conical=c,convex=x,flat=f, knobbed=k,sunken=s
* cap-surface: fibrous=f,grooves=g,scaly=y,smooth=s
* cap-color: brown=n,buff=b,cinnamon=c,gray=g,green=r,pink=p,purple=u,red=e,white=w,yellow=y
* bruises: bruises=t,no=f
* odor: almond=a,anise=l,creosote=c,fishy=y,foul=f,musty=m,none=n,pungent=p,spicy=s
* gill-attachment: attached=a,descending=d,free=f,notched=n
* gill-spacing: close=c,crowded=w,distant=d
* gill-size: broad=b,narrow=n
* gill-color: black=k,brown=n,buff=b,chocolate=h,gray=g, green=r,orange=o,pink=p,purple=u,red=e,white=w,yellow=y
* stalk-shape: enlarging=e,tapering=t
* stalk-root: bulbous=b,club=c,cup=u,equal=e,rhizomorphs=z,rooted=r,missing=?
* stalk-surface-above-ring: fibrous=f,scaly=y,silky=k,smooth=s
* stalk-surface-below-ring: fibrous=f,scaly=y,silky=k,smooth=s
* stalk-color-above-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y
* stalk-color-below-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y
* veil-type: partial=p,universal=u
* veil-color: brown=n,orange=o,white=w,yellow=y
* ring-number: none=n,one=o,two=t
* ring-type: cobwebby=c,evanescent=e,flaring=f,large=l,none=n,pendant=p,sheathing=s,zone=z
* spore-print-color: black=k,brown=n,buff=b,chocolate=h,green=r,orange=o,purple=u,white=w,yellow=y
* population: abundant=a,clustered=c,numerous=n,scattered=s,several=v,solitary=y
* habitat: grasses=g,leaves=l,meadows=m,paths=p,urban=u,waste=w,woods=d


Applying classification algorithms on this dataset could lead to help prevent diseases caused by eating poisonous mushrooms, it can find place in many applications used by the people with this hobby, to help them in ambiguous situations.





This work has been developed using Python 3.6.5 in Jupyter Notebook - web based application that can create interactive documents containing code, images and graphs and text as well. The libraries used in this analisys were:


* pandas - for manipulating in easy way tables and data structures with high performance
* numpy - for manipulating arrays and computation on N-dimensional arrays. It offers also random capabilities
* scikit-learn - it offers tools for data analysis
* plotly - a graph library that creates interactive graphs, way better than matplotlib

In [1]:
# scientific computing libaries
import pandas as pd
import numpy as np

# machine learning libaries
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GridSearchCV, learning_curve
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve, auc, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.pipeline import make_pipeline, Pipeline 

#plot libaries
import plotly
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True) # to show plots in notebook

# offline plotly
from plotly.offline import plot, iplot

# do not show any warnings
import warnings
warnings.filterwarnings('ignore')

SEED = 42 # specify seed for reproducable results
pd.set_option('display.max_columns', None) # to show all the columns


Here some dictionaries are defined to be used later in the classification section for the grid search

In [2]:
RANDOM_FOREST_PARAMS = {
    'clf__max_depth': [25, 50, 75],
    'clf__max_features': ["sqrt"], # max number of features considered for splitting a node: auto is the same as sqrt, and sqrt
#is nearly the same of log2 regarding 21 features as in our dataset (sqrt(21) = 4.3, log2(21) = 4.5)
    'clf__criterion': ['gini', 'entropy'],
    'clf__n_estimators': [100, 300, 500, 1000]
}

DECISION_TREE_PARAMS = {
    'clf__max_depth': [25, 50, 75],
    'clf__max_features': ["sqrt"], # max number of features considered for splitting a node: auto is the same as sqrt, and sqrt
#is nearly the same of log2 regarding 21 features as in our dataset (sqrt(21) = 4.3, log2(21) = 4.5) 
    'clf__criterion': ['gini', 'entropy'],
    'clf__min_samples_split': [6, 10, 14],
}

LOGISTIC_REGRESSION_PARAMS = {
    'clf__solver': ['liblinear'],
    'clf__C': [0.1, 1, 10],
    'clf__penalty': ['l2', 'l1']
}

LDA_PARAMS = {
    'clf__solver': ['lsqr', 'eigen'],
    'clf__shrinkage' : [0.3, 0.5, 0.7, 'auto', None]
}

KNN_PARAMS = {
    'clf__n_neighbors': [5, 15, 25, 35, 45, 55, 65],
    'clf__weights': ['uniform', 'distance'],
    'clf__p': [1, 2, 10]
}

SVM_PARAMS = [
{
    'clf__kernel': ['linear'],
    'clf__C': [0.1, 1, 10],
}, 
{
    'clf__kernel': ['rbf'],
    'clf__C': [0.01, 0.1, 1, 10, 100],
    'clf__gamma': [0.01, 0.1, 1, 10, 100],
}]

# Dataset Exploration



### Overview
In this section, a first look to the dataset is given, trying to get an idea about the conformation of the features and how the different attributes look like

In [3]:
# load the dataset
df = pd.read_csv("/Users/stefa/Desktop/Politecnico/Data Spaces/Tesina/mushrooms.csv")

print("The dataset has %d rows and %d columns." % df.shape)

# check for null values in the dataset
print("There are " + ("some" if df.isnull().values.any() else "no")  + " null/missing values in the dataset.")

The dataset has 8124 rows and 23 columns.
There are no null/missing values in the dataset.


It's positive that there are no null or missing values in the dataset. If it was the case, we would have manage this situation for the next sections.

Now we take a look at the first rows of the dataset

In [4]:
df.head(5)

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,e,e,s,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g


As i wrote in the overview of the dataset, every attribute is cathegorical and not numerical. To achieve statistics and classification, the attributes should be converted to integer values.
To accomplish this, a LabelEncoder is used (actually, it's 23, one for each column) and it is stored in an array for eventually use it in the future.


A LabelEncoder replaces each unique label with a unique integer, storing the dictionary used. I've used this method because every column in this dataset is done in this way, and employing dummy variables on the whol dataset would have been a complete mess.

In [5]:
# create dictionaries to consistently transform string values in numbers
def preprocess_mushrooms(df):
    df_tmp = df.copy()
    lbl_encoders = {}
    
    for col in df_tmp.columns:
        lbl_encoders[col] = LabelEncoder()
        lbl_encoders[col].fit(df_tmp[col])
        df_tmp[col] = lbl_encoders[col].transform(df_tmp[col])
    
    return df_tmp, lbl_encoders

In [6]:
df_preproc, lblenc = preprocess_mushrooms(df)
df_preproc.head(5)

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,0,3,2,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,0,2,2,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,0,2,2,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,0,3,2,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,1,3,2,2,7,7,0,2,1,0,3,0,1


### Statistical overview of attributes

The following statistical measures can be seen for each column using the describe-function of DataFrame of the pandas library:

* count: number of samples
* mean: the mean of this attribute among all samples
* std: the standard deviation of this attribute
* min: the minimal value of this attribute
* 25%: the lower percentile
* 50%: the median
* 75%: the upper percentile
* max: the maximal value of this attribute

In [7]:
df_preproc.describe()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
count,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.0,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000
mean,0.482029,3.348104,1.827671,4.504677,0.415559,4.144756,0.974151,0.161497,0.309207,4.810684,0.567208,1.109798,1.575086,1.603644,5.816347,5.794682,0.0,1.965534,1.069424,2.291974,3.596750,3.644018,1.508616
std,0.499708,1.604329,1.229873,2.545821,0.492848,2.103729,0.158695,0.368011,0.462195,3.540359,0.495493,1.061106,0.621459,0.675974,1.901747,1.907291,0.0,0.242669,0.271064,1.801672,2.382663,1.252082,1.719975
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,3.000000,0.000000,2.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,1.000000,1.000000,6.000000,6.000000,0.0,2.000000,1.000000,0.000000,2.000000,3.000000,0.000000
50%,0.000000,3.000000,2.000000,4.000000,0.000000,5.000000,1.000000,0.000000,0.000000,5.000000,1.000000,1.000000,2.000000,2.000000,7.000000,7.000000,0.0,2.000000,1.000000,2.000000,3.000000,4.000000,1.000000
75%,1.000000,5.000000,3.000000,8.000000,1.000000,5.000000,1.000000,0.000000,1.000000,7.000000,1.000000,1.000000,2.000000,2.000000,7.000000,7.000000,0.0,2.000000,1.000000,4.000000,7.000000,4.000000,2.000000
max,1.000000,5.000000,3.000000,9.000000,1.000000,8.000000,1.000000,1.000000,1.000000,11.000000,1.000000,4.000000,3.000000,3.000000,8.000000,8.000000,0.0,3.000000,2.000000,4.000000,8.000000,5.000000,6.000000


Looking at these numbers we can have an idea about the attributes are spread in the dataset.

We can see, for example, that the attribute "**veil-type**" has mean = 0 and standard deviation = 0. It means that every single value in the dataset has veil-type = 0 so probably it won't be a good predictor for our models.

Anyway, this is not an easy way to visualize these distributions, so let's visualize the data on more useful and clear **graphs**.

First of all, we retrieve the code used by the LabelEncoder to translate the "class" attribute, in order to use this dictionary in the creation of the dataset and in the rest of the work. 

In [8]:
values = lblenc['class'].classes_
keys = lblenc['class'].transform(lblenc['class'].classes_)
for i in range(len(values)):
    if values[i] == 'e':
        values[i] = 'edible'
    elif values[i] == 'p':
        values[i] = 'poisonous'
mush_dict = dict(zip(keys, values))
print(mush_dict)

{0: 'edible', 1: 'poisonous'}


### Visualization

In [9]:
y = df_preproc['class'].value_counts()

colors = plotly.colors.DEFAULT_PLOTLY_COLORS
data = [go.Bar(x=[mush_dict[x] for x in y.index], y=y.values, marker = dict(color = colors[2:2+len(y.index)]))]
layout = go.Layout(
    title="Mushroom Class Distribution",
    autosize=False,
    width=400,
    height=400,
    yaxis=dict(
        title='num samples'
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='basic-bar3')

In [10]:
pois_perc = df_preproc['class'].sum() * 100 / df_preproc['class'].shape[0]
print("Poisonous percentage is %.3f%%." % pois_perc)

Poisonous percentage is 48.203%.


Luckily for us, the dataset is well balanced and it avoids many problems in the later classification. 

In fact, if the dataset was unbalanced (let's say 10% poisonous vs 90% edible) the classifier would be biased on predicting "edible", just because it increments the overall accuracy.
We would have had a lot of false negative and it would have been necessary to use techniques of undersampling to improve performances.

Now, there are some graphs showing the poisonous/edible distribution among all the attributes in the dataset.

In [11]:
pois = df_preproc[df_preproc['class'] == 1]
edib = df_preproc[df_preproc['class'] == 0]

def create_pois_trace(col, visible=False):
    return go.Histogram(
        x=pois[col],
        name='Poisonous',
        marker = dict(color = colors[3]),
        visible=visible,
    )

def create_edib_trace(col, visible=False):
    return go.Histogram(
        x=edib[col],
        name='Edible',
        marker = dict(color = colors[2]),
        visible = visible,
    )

features_not_for_hist = ["class"]
features_for_hist = [x for x in df_preproc.columns if x not in features_not_for_hist]
active_idx = 0
traces_pois = [(create_pois_trace(col) if i != active_idx else create_pois_trace(col, visible=True)) for i, col in enumerate(features_for_hist)]
traces_edib = [(create_edib_trace(col) if i != active_idx else create_edib_trace(col, visible=True)) for i, col in enumerate(features_for_hist)]
data = traces_pois + traces_edib

n_features = len(features_for_hist)
steps = []
for i in range(n_features):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(data)],
        label = features_for_hist[i],
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    step['args'][1][i + n_features] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active = active_idx,
    currentvalue = dict(
        prefix = "Feature: ", 
        xanchor= 'center',
    ),
    pad = {"t": 50},
    steps = steps,
)]

layout = dict(
    sliders=sliders,
    yaxis=dict(
        title='#samples',
        automargin=True,
    ),
)

fig = dict(data=data, layout=layout)

iplot(fig, filename='histogram_slider')

### Feature Analysis

We can clearly see that some characteristics of mushrooms are very specific of poisonous or edible class, for example we can see on the histogram of "odor" that there are some values that are completely unbalanced, showing that they probably will be good predictors.

As we told before, the feature "veil-type" is completely useles as it doesn't add information. So, in the next processing, this column will be removed.

Another interesting graph is the "gill-attachment" one and the "veil-color" one, showing that nearly all of the values for this attributes are the same, so they won't be good predictors, and if we need to get rid of some features to simplify the classification process, this columns would be good candidates.

----

In this dataset, every attribute is cathegorical, so it's not a good idea to search for outliers in the pre-processing phase, so in this work this type of data exploration has been ignored.

Normally, outliers would badly influence the classification, but in this case the attributes are standard and they can't go too far from the mean.

---

An interesting analysis on the feature is the pair-wise correlation: it measures how two attributes are correlated (e.g. if one grows the other grows, or if one become smaller, the other too. This is an ecample of positive correlation). 
It is interesting because it lets us discover which one of the attributes is more correlated to the class "poisonous/edible" and which attributes are highly correlated between them, so that one of the two can be cut off.

To compute this, the Pearson correlation has been used. Between two variables $X$ and $Y$, let $\sigma_X, \sigma_Y$ be the standard deviation of $X, Y$ and $cov(X,Y) = E[(X - E[X])(Y-E[Y])]$.

So we define the Pearson correlation as the following:
$\rho_{X,Y} = \frac{cov(X,Y)}{\sigma_X\sigma_Y}$

The best way to have a first impression of this correlation is to build a heatmap graph, where the color scale on the right can tell us if variance is positive or negative, high or low.

In [12]:
corr = df_preproc.corr()
trace = go.Heatmap(z=corr.values.tolist(), x=corr.columns, y=corr.columns,
                   colorscale=[[1.0, "rgb(49,54,149)"],
                [0.8888888888888888, "rgb(69,117,180)"],
                [0.7777777777777778, "rgb(116,173,209)"],
                [0.6666666666666666, "rgb(171,217,233)"],
                [0.5555555555555556, "rgb(224,243,248)"],
                [0.4444444444444444, "rgb(254,224,144)"],
                [0.3333333333333333, "rgb(253,174,97)"],
                [0.2222222222222222, "rgb(244,109,67)"],
                [0.1111111111111111, "rgb(215,48,39)"],
                [0.0, "rgb(165,0,38)"]]
                  )
data=[trace]
layout = go.Layout(
    title='Heatmap of pairwise correlation of the columns',
    autosize=False,
    width=850,
    height=700,
    yaxis=go.layout.YAxis(automargin=True),
    xaxis=dict(tickangle=40),
    margin=go.layout.Margin(l=0, r=200, b=200, t=80)
)


fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='labelled-heatmap1')

We can see high positive correlation between "veil-color" and "gill-attachment". This is a false positive correlation, it happens just because both of the attributes have nearly always the same values, so it seems that they are correlated.
Actually, we can see how the veil-type doesn't show any correlation due to his standard deviation $\sigma = 0$ that makes Pearson correlation go to infinity

In order to reduce the dimensionality of the dataset, it could be useful to apply a clustering algorithm to identify features near between them, so we can remove duplicates. For this, we conduct a clustering of the features using agglomerative hierarchical clustering with average linkage. This method starts by creating one cluster for each feature and by computing the pairwise distance/dissimilarity/similarity between all the clusters, which in our case is the correlation. Then it select the two clusters with the highest average correlation to be merged. In the next iteration, the next pair of clusters is selected to be merged. This process is repeated until we end up with one cluster.

To visualize the clustering process a dendrogram is shown in the following:

In [13]:
#had to remove veil-type to obtain clustering because it's all zero and it can't do the dendrogram showing the clustering
df_preproc=df_preproc.drop(["veil-type"], axis=1)

from scipy.cluster import hierarchy as hc
X = np.random.rand(10, 10)
names = df_preproc.columns
inverse_correlation = 1 - abs(df_preproc.corr())
fig = ff.create_dendrogram(inverse_correlation.values, orientation='left', labels=names, colorscale=colors, linkagefun=lambda x: hc.linkage(x, 'average'))
fig['layout'].update(dict(
    title="Dendrogram of clustering the features according to correlation",
    width=800, 
    height=600,
    margin=go.layout.Margin(l=180, r=50),
    xaxis=dict(
        title='distance',
    ),
    yaxis=dict(
        title='features',
        automargin=True,
    ),
))
iplot(fig, filename='dendrogram_corr_clustering')

As we can see, effectively gill-attachment and veil-color are very near in terms of distance. In order to reduce the dimensionality, we can think of deleting one of them. "gill-attachment" is a better candidate, because looking in the graphs showing the distribution poisonous/edible, in both of the possible cases the distribution is similar to the poisonous/edible on the whole dataset, instead in the "veil-color" graph, some values have a distribution poisonous/edible more unbalanced (although it's a very small portion of data) so it would be more useful.

We prepare the dataset for the analysis, splitting it into two partes: the target column and other predictors. We standardize also the data to avoid higher impact of features with higher absolute values. This wouldn't be necessary for decision tree or random forest, but it's not a problem if the data is standardized, so it's done at this level.

In [14]:
features_to_remove = ["veil-type", "gill-attachment"]

# split the dataset in feature and target label
df_y = df_preproc["class"]
df_X = df_preproc.drop(["class"], axis=1)

cols_to_keep = [x for x in df_X.columns if x not in features_to_remove]

# normalize the dataset
df_X_normed = (df_X - df_X.mean()) / df_X.std()

# Principal Component Analysis

The main idea of Principal Component Analysis (PCA) is to reduce the dimensionality of a dataset consisting of many attributes, while retaining the variation present in the dataset up to the maximun extent. This is achieved transforming the variables to a new set of variables, which are known as the principal components (simply, PCs) and are orthogonal, ordered such that the retention of variation present in the original attributes decreases as we move down in the order. So, in this way, the 1st principal component retains maximum variation that was present in the original components. The principal components are the eigenvectors of a covariance matrix, and hence they are orthogonal.

Technically, a pricipal component can be defined as a linear combination of optimally-weighted observed variables. To achieve PCA here are the steps:

#### Step 1: Normalize the data
---
First step is to normalize the data so that PCA works properly. This is done by subtracting the respective means from the numbers in the respective column. This produces a dataset whose mean is zero
  
#### Step 2: Calculate the covariance matrix
---
This matrix is computed with the covariance of every couple of feature. For example, in a case of a dataset 2-dimensional, this will result in a 2x2 covariance matrix:
  
<center>$Matrix(Covariance) = \begin{bmatrix} Var[X_1] & Cov[X_1,X_2] \\ Cov[X_2,X_1] & Var[X_2] \end{bmatrix}$</center>
    
Note that $Var[X_1] = Cov[X_1,X_1]$ and $Var[X_2] = Cov[X_2,X_2]$
  
#### Step 3: Calculate eigenvalues and eigenvectors
---
Next step is to calculate eigenvalues and eigenvectors for the covariance matrix. The same is possible because it is a square matrix. $\lambda$ is an eigenvalue for a matrix $A$ if it is a solution of the characteristic equation:

<center>$det(\lambda I - A) = 0$</center>

Where I is the identity matrix of the same dimensions as A (required for subtraction) and '$det$' is the determinant of the matrix. For each eigenvalue $\lambda$, a corresponding eigenvector $v$ can be found by solving:

<center>$(\lambda I - A)v = 0$</center>

#### Step 4: Choosing components
---
We order the eigenvalues from largest to smallest so that it gives us the components in order of significance. We create then a feature vector that contains the components we want. For example, if we want to reduce our dataset of 21 features to 5, we choose the 5 eigenvectors corresponding to the 5 highest eigenvalues and compose the feature vector like this:

<center>$ FeatureVector = (eig_1, eig_2, eig_3, eig_4, eig_5)$</center>

#### Step 5: Forming Principal Components
---
This is the final step where we actually form the principal components. In order to accomplish this, we take the transpose of the feature vector and multiply it with the traspose of the normalized dataset
<center>$ NewData = FeatureVector^T x NormData^T$</center>


We apply PCA to the dataset in order to reduce the number of features

In [15]:
# calculate the principal components
pca = PCA(random_state=SEED)
df_X_pca = pca.fit_transform(df_X_normed)

tot = sum(pca.explained_variance_) # total explained variance of all principal components
var_exp = [(i / tot) * 100 for i in sorted(pca.explained_variance_, reverse=True)] # individual explained variance
cum_var_exp = np.cumsum(var_exp) # cumulative explained variance

In the following graph is plotted the relation between number of PCs and variance explained, in order to choose wisely how many components we use.

In [16]:
trace_cum_var_exp = go.Bar(
    x=list(range(1, len(cum_var_exp) + 1)), 
    y=var_exp,
    name="individual explained variance",
)
trace_ind_var_exp = go.Scatter(
    x=list(range(1, len(cum_var_exp) + 1)),
    y=cum_var_exp,
    mode='lines+markers',
    name="cumulative explained variance",
    line=dict(
        shape='hv',
    ))
data = [trace_cum_var_exp, trace_ind_var_exp]
layout = go.Layout(
    title='Individual and Cumulative Explained Variance',
    autosize=True,
    yaxis=dict(
        title='percentage of explained variance',
    ),
    xaxis=dict(
        title="principal components",
        dtick=1,
    ),
    legend=dict(
        x=0,
        y=1,
    ),
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='basic-bar')

We can clearly see that the first 5 components (about 1/4 of the attributes of the dataset) explain more then 60% of the variance. Using 9 PCs we are over 80% of variance explained, and with 15 PCs we are nearly to 100% variance explained (95.44%).

It's interesting to test a dataset reduced to 15, 9 and 5 PC to see how bad or good the classifier performs on reduced dimensions, to understand if even five components with 60% of variance explained are sufficient to create a good model.

In [17]:
df_X_red = df_X_normed[cols_to_keep]

df_X_pca = np.dot(df_X_normed.values, pca.components_[:5,:].T)
df_X_5PC = pd.DataFrame(df_X_pca, columns=["PC#%d" % (x+1) for x in range(5)])
df_X_pca = np.dot(df_X_normed.values, pca.components_[:9,:].T)
df_X_9PC = pd.DataFrame(df_X_pca, columns=["PC#%d" % (x+1) for x in range(9)])
df_X_pca = np.dot(df_X_normed.values, pca.components_[:15,:].T)
df_X_15PC = pd.DataFrame(df_X_pca, columns=["PC#%d" % (x+1) for x in range(15)])

### Comparison for classification of differently preprocessed datasets 

We would like to show in this section how well a classifier performs with our differently preprocessed dataset, in order to get how many components are needed to achieve a good result.
It has been used the Random Forest classifier because it performs very well and it can give us a precise idea on how well the preprocessed datasets are modelling the classification problem.

We test the classifier on the following versions of the mushroom dataset:
+ Full dataset
+ Dataset without columns deleted due to low variance and correlation
+ Datased reduced to 15 principal components after applying PCA
+ Dataset reduced to 9 principal components after applying PCA
+ Dataset reduced to 5 principal components after applying PCA

For the evaluation of the model we keep a validation set with 20% of all the data

In [18]:
X_train, X_test, y_train, y_test = train_test_split(df_X_normed, df_y, test_size=0.25, random_state = SEED)
X_train_red, X_test_red, y_train_red, y_test_red = train_test_split(df_X_red, df_y, test_size=0.25, random_state = SEED)
X_train_5PC, X_test_5PC, y_train_5PC, y_test_5PC = train_test_split(df_X_5PC, df_y, test_size=0.25, random_state = SEED)
X_train_9PC, X_test_9PC, y_train_9PC, y_test_9PC = train_test_split(df_X_9PC, df_y, test_size=0.25, random_state = SEED)
X_train_15PC, X_test_15PC, y_train_15PC, y_test_15PC = train_test_split(df_X_15PC, df_y, test_size=0.25, random_state = SEED)

In [19]:
print("Shape of the full train dataset:", X_train.shape)
print("Shape of the train dataset with reduced features", X_train_red.shape)
print("Shape of the transformed train dataset using the first 15 Principal Components", X_train_15PC.shape)
print("Shape of the transformed train dataset using the first 9 Principal Components", X_train_9PC.shape)
print("Shape of the transformed train dataset using the first 5 Principal Components", X_train_5PC.shape)

Shape of the full train dataset: (6093, 21)
Shape of the train dataset with reduced features (6093, 20)
Shape of the transformed train dataset using the first 15 Principal Components (6093, 15)
Shape of the transformed train dataset using the first 9 Principal Components (6093, 9)
Shape of the transformed train dataset using the first 5 Principal Components (6093, 5)


-----

For the evaluation, the main evaluation metric that has been chosen is **F1-Score**. It is defined in the following, and it is based on this measures:
 + $TP$ = *True Positive* - number of samples where the prediction is positive and the true label is positive
 + $FP$ = *False Positive* - number of samples where the prediction is positive but the true label is negative
 + $TN$ = *True Negative* - number of samples where the prediction is negative and the true label is negative
 + $FN$ = *False Negative* - number of samples where the prediction is negative but the true label is positive
 
Said that, we can define the following:

<center>$precision=\frac{TP}{TP+FP}$</center>

<center>$recall = \frac{TP}{TP+FN}$</center>

Then the F1-Score is given by:

<center>$F_1=2\frac{precision * recall}{precision + recall}$</center>

This evaluation metric is very useful in case of unbalanced classes, where the precision itself can poorly describe how well the model performs on the minority class. In this case, our classes are balanced so it wouldn't be needed, but it's a good measure, and it fits very well in our problem. It's mainly because if we are trying to classify a mushroom poisonous or edible, we are interested whether our mothel gives a lot of false negatives, that could be a serious problem for the people using this application. So, the F1-Score is used in this test and in the classification section as the main metric for evaluate the models.

In [20]:
# prints the best grid search scores along with their parameters.
def print_best_grid_search_scores_with_params(grid_search, n=10):
    if not hasattr(grid_search, 'best_score_'):
        raise KeyError('grid_search is not fitted.')
    print("Best grid scores on validation set:")
    indexes = np.argsort(grid_search.cv_results_['mean_test_score'])[::-1][:n]
    means = grid_search.cv_results_['mean_test_score'][indexes]
    stds = grid_search.cv_results_['std_test_score'][indexes]
    params = np.array(grid_search.cv_results_['params'])[indexes]
    for mean, std, params in zip(means, stds, params):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
        
def do_gridsearch_with_cv(clf, params, X_train, y_train, cv):
    pipeline = Pipeline([('clf', clf)])    
    gs = GridSearchCV(pipeline, params, cv=kf, n_jobs=-1, scoring='f1', return_train_score=True)
    gs.fit(X_train, y_train)
    return gs

def score_on_test_set(clfs, datasets):
    scores = []
    for c, (X_test, y_test) in zip(clfs, datasets):
        scores.append(c.score(X_test, y_test))
    return scores

Each classifier is defined by some **hyperparameters**; they are responsible of the of how well the classifier performs on the data, and can be tuned to find the best combination of them for each classifier. At the beginning of this notebook, a set of hyperparameters for every classifier are defined, and they are used in a grid-search fashion to find the combination that achieves the best score.

In order to evaluate each classifier in a high reliable way, it has been used a method called **K-fold Cross Validation**. 
This method goal is to use part of the train set as a validation set, splitting it in $K$ equal parts. The classifier is then trained on a train set composed of $K-1$ parts and tested on the part held out. This process is repeated $K$ times, leaving every time out a different part of the train set.
In this notebook, we use **"Stratified K-Fold"**, it means that in each subset we have the same number of samples from the two classes.
The score is then computed on the subset not used for the training phase.

The final score is then obtained by averaging the score of each of the classifier $C_k$ obtained in the $K$ iterations

<center>$Score_{CV(K)} = \frac{1}{K}\sum_{k=1}^{K} F_1Score(Ck)$</center>

----

### Evaluation

A very common and useful method to do this hyperparameters analysis is **grid search**. It simply consists of creating a table with the result of each combination of parameters, and choosing the combination that performs better. 

In python a useful library is provided to achieve this goal - **GridSearchCV** - from the package of SciKit Learn sklearn.model_selection. It receives as arguments a classifier, a dictionary containing all the hyperparameters with corresponding values and a k-fold object. It is very common choosing $k=5$ or $k=10$, but in our case it hase been chosen $k=5$ to achieve good precision in the score but also good performances

In [21]:
kf = StratifiedKFold(n_splits=5, random_state=SEED)
clf_rf = RandomForestClassifier(random_state=SEED)

In [22]:
%%time
gs_full = do_gridsearch_with_cv(clf_rf, RANDOM_FOREST_PARAMS, X_train, y_train, kf)
gs_red = do_gridsearch_with_cv(clf_rf, RANDOM_FOREST_PARAMS, X_train_red, y_train_red, kf)
gs_15PC = do_gridsearch_with_cv(clf_rf, RANDOM_FOREST_PARAMS, X_train_15PC, y_train_15PC, kf)
gs_9PC = do_gridsearch_with_cv(clf_rf, RANDOM_FOREST_PARAMS, X_train_9PC, y_train_9PC, kf)
gs_5PC = do_gridsearch_with_cv(clf_rf, RANDOM_FOREST_PARAMS, X_train_5PC, y_train_5PC, kf)

gss_raw = [gs_full, gs_red, gs_15PC, gs_9PC, gs_5PC]

test_results_raw = score_on_test_set(gss_raw, [(X_test, y_test), (X_test_red, y_test_red), (X_test_15PC, y_test_15PC), (X_test_9PC, y_test_9PC), (X_test_5PC, y_test_5PC)])

Wall time: 17min 56s


In [23]:
dataset_strings = ["full dataset", "data set with reduced features", "dataset with first 15 principal components", "dataset with first 9 principal components", "dataset with first 5 principal components"]

result_strings = dict()
for ds, res in zip(dataset_strings, test_results_raw):
    string = "%.3f" % res + "     " + ds + " " 
    result_strings[string] = res

result_strings = sorted(result_strings.items(), key=lambda kv: kv[1], reverse=True)
print("F1 score  dataset")
for k, _ in result_strings:
    print(k)

F1 score  dataset
1.000     full dataset 
1.000     data set with reduced features 
1.000     dataset with first 15 principal components 
1.000     dataset with first 9 principal components 
0.996     dataset with first 5 principal components 


### Comparison

The classifier performs very well on our dataset. We can see that there is no difference in score between the full dataset and the dataset without the column with low variance/high correlation to other features: in each case, it achieves a perfect F1-Score of 1.

It's also very interesting noticing that with the first 15 PCs and even the first 9 PCs the classifier obtains again a perfect score. It means that if we ever need to reduce dimensionality we can obtain a very high performance with only 9 PCs. The same can be noticed with 5 PCs, that compared to our 21 features  is a very small number, but also in this case the classifier achieves an impressing score of 0.997, making clear that even with only 5 principal components we can model very well our dataset for the classification problem.

We can be scure that these models aren't overfitted because of the usage of K-Fold method

--------

# Classification

### Regression vs Classification

Variables can be characterized as either *quantitative* or *qualitative* (also known as *categorical*). Quantitative variables take on numerical values (e.g. person's age, income, value of a house); in contrast, qualitative variablestake on values of K different classes, or categories (e.g. person's gender - male or female - or brand of product purchased - brand A, B or C).

Problems with a qualitative response are referred to as **regression** problems, while those involving a qualitative response are often referred to as **classification** problems.

However, the distincion is not so strict, for example *Logistic Regression* is typically used with a qualitative (two-class, or binary) response, and so it is often used as a classification method, but since it estimates class probabilities, it can be thought of as a regression method as well. Equally, some statistical methods (e.g. K-Nearest Neighbors) can be used in the case of either quantitative or qualitative responses

---

In our case, the goal is to perform a *binary classifiation* on the variable "class" of the dataset - poisonous vs edible. This is a case of a qualitative response, so classification methods are employed, but because it's a two-class problem, also Logistic Regression has been used.

In [24]:
df_X_pca = np.dot(df_X_normed.values, pca.components_[:2,:].T)
df_X_2PC = pd.DataFrame(df_X_pca, columns=["PC#%d" % (x+1) for x in range(2)])

def get_color_with_opacity(color, opacity):
    return "rgba(" + color[4:-1] + ", %.2f)" % opacity

# partially based on https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html
def plot_learning_curve(estimator, title, X, y, cv=None, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring="f1", random_state=SEED)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    trace1 = go.Scatter(
        x=train_sizes, 
        y=train_scores_mean - train_scores_std, 
        showlegend=False,
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = get_color_with_opacity(colors[7], 0.4),
        ),
    )
    trace2 = go.Scatter(
        x=train_sizes, 
        y=train_scores_mean + train_scores_std, 
        showlegend=False,
        fill="tonexty",
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = get_color_with_opacity(colors[7], 0.4),
        ),
    )
    trace3 = go.Scatter(
        x=train_sizes, 
        y=train_scores_mean, 
        showlegend=True,
        name="Train score",
        line = dict(
            color = colors[7],
        ),
    )
    
    trace4 = go.Scatter(
        x=train_sizes, 
        y=test_scores_mean - test_scores_std, 
        showlegend=False,
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = get_color_with_opacity(colors[8], 0.4),
        ),
    )
    trace5 = go.Scatter(
        x=train_sizes, 
        y=test_scores_mean + test_scores_std, 
        showlegend=False,
        fill="tonexty",
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = get_color_with_opacity(colors[8], 0.4),
        ),
    )
    trace6 = go.Scatter(
        x=train_sizes, 
        y=test_scores_mean, 
        showlegend=True,
        name="Test score",
        line = dict(
            color = colors[8],
        ),
    )
    
    data = [trace1, trace2, trace3, trace4, trace5, trace6]
    layout = go.Layout(
        title=title,
        autosize=True,
        yaxis=dict(
            title='F1 Score',
        ),
        xaxis=dict(
            title="#Training samples",
        ),
        legend=dict(
            x=0.8,
            y=0,
        ),
    )
    fig = go.Figure(data=data, layout=layout)
    return iplot(fig, filename=title)
def plot_feature_importance(feature_importance, title):
    trace1 = go.Bar(
        x=feature_importance[:, 0],
        y=feature_importance[:, 1],
        marker = dict(color = colors[0]),
        name='feature importance'
    )
    data = [trace1]
    layout = go.Layout(
        title=title,
        autosize=True,
        margin=go.layout.Margin(l=50, r=100, b=150),
        xaxis=dict(
            title='feature',
            tickangle=30
        ),
        yaxis=dict(
            title='feature importance',
            automargin=True,
        ),
    )
    fig = go.Figure(data=data, layout=layout)
    return iplot(fig, filename=title)

### Logistic Regression

Logistic Regression is the appropriate regression analysis to conduct when the dependent variable is binary. The problem with other types of regression (e.g. Linear Regression) is that if a straight line is fit to a binary response coded as 0 or 1, we can always predict $p(X) < 0$ for some values of $X$ and $p(X) > 1$ for others.

To avoid this problem, we must model $p(X)$ using a function that gives outputs between 0 and 1 for all values of $X$. In logistic regression we use the *logistic function* defined as the following:

<center>$p(X) = \frac{e^{\beta_0+X^T\beta}}{1 + e^{\beta_0+X^T\beta}}$</center>

Where $p(X)$ means the probability of sample $X$ to be in class 1, $X$ the $n$-dimensional array of features and $\beta_0, \beta = (\beta_1, ..., \beta_n)^T$ the parameters of the model.

The hyperparameters of logistic regression (passed as arguments to LogisticRegression of sklear.linear_model) include the following ones:
 + penalty: the norm used for penalization (default='l2')
 + C: the inverse of the regularization strength (default=1.0)

In [25]:
%%time
clf_lr = LogisticRegression(random_state=SEED)
gs_lr = do_gridsearch_with_cv(clf_lr, LOGISTIC_REGRESSION_PARAMS, X_train, y_train, kf)

Wall time: 3.44 s


In [26]:
print_best_grid_search_scores_with_params(gs_lr)

Best grid scores on validation set:
0.962 (+/-0.015) for {'clf__C': 10, 'clf__penalty': 'l1', 'clf__solver': 'liblinear'}
0.962 (+/-0.013) for {'clf__C': 10, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}
0.960 (+/-0.015) for {'clf__C': 1, 'clf__penalty': 'l1', 'clf__solver': 'liblinear'}
0.951 (+/-0.010) for {'clf__C': 1, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}
0.945 (+/-0.006) for {'clf__C': 0.1, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}
0.944 (+/-0.008) for {'clf__C': 0.1, 'clf__penalty': 'l1', 'clf__solver': 'liblinear'}


In [27]:
gs_lr_score = gs_lr.score(X_test, y_test)
y_pred_lr = gs_lr.predict(X_test)
cm_lr = confusion_matrix(y_test, y_pred_lr)
cm_lr = cm_lr.astype('float') / cm_lr.sum(axis=1)[:, np.newaxis] # normalize the confusion matrix
cm_df = pd.DataFrame(cm_lr.round(3), index=["true edible", "true poisonous"], columns=["predicted edible", "predicted poisonous"])
cm_df

,predicted edible,predicted poisonous
true edible,0.961,0.039
true poisonous,0.026,0.974


Here the normalized confusion matrix is shown, as well as the result of the grid search on the top of it. The confusion matrix is a useful tool to see the accuracy of the model in terms of False Positive or False Negative. 

It shows that this model fits well our data, achieving an overall F1-Score of 0.962 and not showing particular bias towards any of the class (both False Positive and False Negative - the one we are more intrested in - are very low and similar).

In [28]:
plot_learning_curve(gs_lr.best_estimator_, "Learning Curve of Logistic Regression", X_train, y_train, cv=5)

It's not evident a clear tendency in the learning curve of Logistic Regression, but we can see that it tends to increase with the number of training samples. The most interesting thing we can understand from this plot is that the score on train set and test set at the end of the training phase is very similar, and this is a sure confirm that this model is not overfitting the training data.

### Linear Discriminant Analysis [LDA]

Instead of Logistic Regression, we can consider an alternative and less direct approach to estimate this probabilities; we model the distribution of the predictors $X$ separately in each of the response classes (given $Y$), and then use Bayes' theorem to flip these around into estimates for $P(Y=k|X=x)$.

One of the reason to use another method is that when the classes are well-separated, the parameter estimates for the logistic regression model are unstable, while LDA does not soffer from this problem.

LDA is an approach to reduce dimensionality projecting a dataset onto a lower-dimensional space with good class-separability in order to avoid overfitting (curse of dimensionality) and also reduce computational costs, conceptually very similar to PCA. PCA can be described as an "unsupervised" algorithm, since it ignores class labels and it finds the directions (PCs) that maximize the variance in a dataset. In contrast to PCA, LDA is "supervised" and computes the directions ("linear discriminants") that will represent the axes that maximize the separation between multiple classes.

For classification, is used in this way. Instead of estimating $P(Y|X)$ we will estimate:
 + $P(X|Y)$: given the response, what is the distribution of the inputs
 + $P(Y)$: how likely are each of the categories

Then, we use *Bayes' theorem* to obtain the estimate:
<center>$P(Y=k | X=x) = \frac{P(X=x | Y=k)P(Y=k)}{P(X=x)}$</center>
<br><center>or</center><br>
<center>$P(Y=k | X=x) = \frac{P(X=x | Y=k)P(Y=k)}{\sum_{j} P(X=x | Y=j)P(Y=j)}$</center>

The hyperparameters of linear discriminant analysis (passed as arguments to LinearDiscriminantAnalysis of sklear.discriminant_analysis) include the following ones:

+ solver: least squares solution ('lsqr') or eigenvalue decomposition ('eigen')
+ shrinkage: tool to improve estimation of covariance matrices in situations where the number of training samples is small compared to the number of features. It can be a float between 0 and 1, or `None` or 'auto' 

In [29]:
%%time
clf_lda = LDA()
gs_lda = do_gridsearch_with_cv(clf_lda, LDA_PARAMS, X_train, y_train, kf)

Wall time: 960 ms


In [30]:
print_best_grid_search_scores_with_params(gs_lda)

Best grid scores on validation set:
0.941 (+/-0.009) for {'clf__shrinkage': None, 'clf__solver': 'lsqr'}
0.940 (+/-0.007) for {'clf__shrinkage': 'auto', 'clf__solver': 'lsqr'}
0.940 (+/-0.008) for {'clf__shrinkage': None, 'clf__solver': 'eigen'}
0.939 (+/-0.006) for {'clf__shrinkage': 'auto', 'clf__solver': 'eigen'}
0.906 (+/-0.006) for {'clf__shrinkage': 0.3, 'clf__solver': 'lsqr'}
0.905 (+/-0.006) for {'clf__shrinkage': 0.3, 'clf__solver': 'eigen'}
0.892 (+/-0.008) for {'clf__shrinkage': 0.5, 'clf__solver': 'lsqr'}
0.891 (+/-0.008) for {'clf__shrinkage': 0.5, 'clf__solver': 'eigen'}
0.884 (+/-0.007) for {'clf__shrinkage': 0.7, 'clf__solver': 'lsqr'}
0.883 (+/-0.007) for {'clf__shrinkage': 0.7, 'clf__solver': 'eigen'}


In [31]:
gs_lda_score = gs_lda.score(X_test, y_test)
y_pred_lda = gs_lda.predict(X_test)
cm_lda = confusion_matrix(y_test, y_pred_lda)
cm_lda = cm_lda.astype('float') / cm_lda.sum(axis=1)[:, np.newaxis] # normalize the confusion matrix
cm_df = pd.DataFrame(cm_lda.round(3), index=["true edible", "true poisonous"], columns=["predicted edible", "predicted poisonous"])
cm_df

,predicted edible,predicted poisonous
true edible,0.954,0.046
true poisonous,0.059,0.941


Here the normalized confusion matrix is shown, as well as the result of the grid search on the top of it.

It shows that this model fits well our data, achieving an overall F1-Score of 0.941 and not showing particular bias towards any of the class (both False Positive and False Negative - the one we are more intrested in - are very low and similar).

In [32]:
plot_learning_curve(gs_lda.best_estimator_, "Learning Curve of LDA", X_train, y_train, cv=5)

We can see how the score on the test set is slowly increasing with the number of samples, while the train score decrease. This is probably due to the fact that at the beginning in the train phase it's easy to overfit with few samples, but increasing the number of samples the LDA algorithm computes a more accurate probability and achieve a better score on the test set.

### K-Nearest Neighbors

K-Nearest Neighbors (or *KNN*) is one of the simplest and most used learning algorithm for classification and it is a non-parametric algorithm (it means that it does not make any assumptions on the underlying data distribution).

It's called a "lazy" algorithm because it does not use training data points to do any generalization (lack of generalization means that KNN keeps all of the training data), so the training phase is pretty fast. The KNN algorithm is based on feature similarity: how closely features resemble our training set determines how we classify a given data point.

Its functioning is quite easy: it considers the $K$ closest training examples to the point of interest for predicting its class, and this is done by a simple majority vote over the $K$ nearest points.

The hyperparameters of KNN (passed as arguments to KNeighborsClassifier of sklear.neighbors) include the following ones:

+ n_neighbors: corresponds to K, the number of nearest neighbors considered for prediction (default=5)
+ weights: 
   - 'uniform': all the neighbors have the same weight for voting (default)
   - 'distance': the votes of the neighbors are weighted by the inverse of the distance
+ p: power parameter for the Minkowski metric (default=2)

In [33]:
%%time
clf_knn = KNeighborsClassifier()
gs_knn = do_gridsearch_with_cv(clf_knn, KNN_PARAMS, X_train, y_train, kf)

Wall time: 6min 52s


In [34]:
print_best_grid_search_scores_with_params(gs_knn)

Best grid scores on validation set:
1.000 (+/-0.000) for {'clf__n_neighbors': 5, 'clf__p': 1, 'clf__weights': 'uniform'}
1.000 (+/-0.000) for {'clf__n_neighbors': 5, 'clf__p': 1, 'clf__weights': 'distance'}
1.000 (+/-0.000) for {'clf__n_neighbors': 15, 'clf__p': 1, 'clf__weights': 'distance'}
1.000 (+/-0.001) for {'clf__n_neighbors': 25, 'clf__p': 1, 'clf__weights': 'distance'}
1.000 (+/-0.001) for {'clf__n_neighbors': 5, 'clf__p': 2, 'clf__weights': 'uniform'}
1.000 (+/-0.001) for {'clf__n_neighbors': 5, 'clf__p': 2, 'clf__weights': 'distance'}
1.000 (+/-0.001) for {'clf__n_neighbors': 35, 'clf__p': 1, 'clf__weights': 'distance'}
1.000 (+/-0.001) for {'clf__n_neighbors': 55, 'clf__p': 1, 'clf__weights': 'distance'}
1.000 (+/-0.001) for {'clf__n_neighbors': 65, 'clf__p': 1, 'clf__weights': 'distance'}
0.999 (+/-0.001) for {'clf__n_neighbors': 45, 'clf__p': 1, 'clf__weights': 'distance'}


In [35]:
gs_knn_score = gs_knn.score(X_test, y_test)
y_pred_knn = gs_knn.predict(X_test)
cm_knn = confusion_matrix(y_test, y_pred_knn)
cm_knn = cm_knn.astype('float') / cm_knn.sum(axis=1)[:, np.newaxis] # normalize the confusion matrix
cm_df = pd.DataFrame(cm_knn.round(3), index=["true edible", "true poisonous"], columns=["predicted edible", "predicted poisonous"])
cm_df

,predicted edible,predicted poisonous
true edible,1.0,0.0
true poisonous,0.0,1.0


Here the normalized confusion matrix is shown, as well as the result of the grid search on the top of it.

It shows that this model fits really well our data, achieving an overall F1-Score of 1.00 and not showing any bias towards any of the class.

In [36]:
plot_learning_curve(gs_knn.best_estimator_, "Learning Curve of KNN", X_train, y_train, cv=5)

It can be seen from the graph that the test score of the KNN increases with the number of samples, reaching the maximum of 1.00. It does make sense because increasing the training points permits the model to make more accurate predictions on the test points. It shows that the KNN is not overfitting the train data, even if it reaches a score of 1.00 on the train set.

### Decision Tree

A Decision Tree is a classifier that is based on a tree-structure to predict the output label. It consists of several splits, which determine for a input sample the predicted class (which is a leaf in the tree).

The construction of the decision trees is made with a greedy approach; it's done because finding the theoretical minimum vould be NP-hard and the number of partitions has a factorial growth.<br>A greedy top-down algorithm is used to create the tree: each step a variable is chosen, the one that best splits the set of items. It is chosen thanks to some metrics that generally measure the homogeneity of the target variable within the subsets (for example Entropy measure).<br>In this analysis, the following metrics have been used:

- Gini Index: <br>
<center>$G = \sum_{k=1}^{K} p_{mk}(1-p_{mk})$</center><br>where $K$ represents number of classes while $p_{mk}$ represents the proportion of training observations in the $m$th region that are from the $k$th class. The Gini index is referred to as a measure of node *purity* - a small value indicates that a node contains predominantly observations from a single class.
- Information Gain: it is based on the decrease in entropy after a data-set split on an attribute. Constructing a decision tree is all about  finding attribute that returns the highest information gain (i.e. the most homogeneous branches). The entropy is defined as $D(p) = - \sum_{k=1}^{K} p_{mk}\log p_{mk}$. Then we define the information gain to split $n$ samples in node $p$ into k partitions, where $n_i$ is the number of samples in partition $i$ as <br><center>$IG = D(p) - \sum_{i=1}^{k} \frac{n_i}{n}D(i)$</center>

The hyperparameters of a Decision Tree (passed as arguments to DecisionTreeClassifier of sklearn.tree) include the following ones:

+ criterion: the criterion which is responsible of the decision to split a node (default='gini')
+ max_depth: the maximum depth of the tree (default=`None`)
+ min_samples_split: the minimum number of samples in a node to be considered for further splitting (default=2)

In [37]:
%%time
clf_dt = DecisionTreeClassifier(random_state=SEED)
gs_dt = do_gridsearch_with_cv(clf_dt, DECISION_TREE_PARAMS, X_train, y_train, kf)

Wall time: 544 ms


In [38]:
print_best_grid_search_scores_with_params(gs_dt)

Best grid scores on validation set:
1.000 (+/-0.000) for {'clf__criterion': 'entropy', 'clf__max_depth': 75, 'clf__max_features': 'sqrt', 'clf__min_samples_split': 14}
1.000 (+/-0.000) for {'clf__criterion': 'entropy', 'clf__max_depth': 75, 'clf__max_features': 'sqrt', 'clf__min_samples_split': 6}
1.000 (+/-0.000) for {'clf__criterion': 'entropy', 'clf__max_depth': 50, 'clf__max_features': 'sqrt', 'clf__min_samples_split': 14}
1.000 (+/-0.000) for {'clf__criterion': 'entropy', 'clf__max_depth': 50, 'clf__max_features': 'sqrt', 'clf__min_samples_split': 10}
1.000 (+/-0.000) for {'clf__criterion': 'entropy', 'clf__max_depth': 50, 'clf__max_features': 'sqrt', 'clf__min_samples_split': 6}
1.000 (+/-0.000) for {'clf__criterion': 'entropy', 'clf__max_depth': 25, 'clf__max_features': 'sqrt', 'clf__min_samples_split': 14}
1.000 (+/-0.000) for {'clf__criterion': 'entropy', 'clf__max_depth': 25, 'clf__max_features': 'sqrt', 'clf__min_samples_split': 10}
1.000 (+/-0.000) for {'clf__criterion': 'e

In [39]:
gs_dt_score = gs_dt.score(X_test, y_test)
y_pred_dt = gs_dt.predict(X_test)
cm_dt = confusion_matrix(y_test, y_pred_dt)
cm_dt = cm_dt.astype('float') / cm_dt.sum(axis=1)[:, np.newaxis] # normalize the confusion matrix
cm_df = pd.DataFrame(cm_dt.round(3), index=["true edible", "true poisonous"], columns=["predicted edible", "predicted poisonous"])
cm_df

,predicted edible,predicted poisonous
true edible,1.0,0.0
true poisonous,0.0,1.0


Here the normalized confusion matrix is shown, as well as the result of the grid search on the top of it.

It shows that this model fits really well our data, achieving an overall F1-Score of 1.00 and not showing any bias towards any of the class.

In [40]:
plot_learning_curve(gs_dt.best_estimator_, "Learning Curve of the Decision Tree", X_train, y_train, cv=5)

It can be seen from the graph that the test score of the Decison Tree increases with the number of samples, reaching the maximum of 1.00. It shows that the Decision Tree is not overfitting the train data, even if it reaches very early a score of 1.00 on the train set.
    
The DecisionTreeClassifier class of sklearn has also a feature very useful, it can compute the importance value for each feature. This is achieved by weighting the decrease of impurity at each split by the probability of reaching this node for each node which split involves the feature of interest. Then these weighted decreases of impuritu are summed up for each feature and this gives the feature importance.

In [41]:
feature_importance = np.array(sorted(zip(X_train.columns, gs_dt.best_estimator_.named_steps['clf'].feature_importances_), key=lambda x: x[1], reverse=True))
plot_feature_importance(feature_importance, "Feature importance in the decision tree")

We can see how the most important features for the Decision Tree are the "spore-print-color" and the "stalk-shape". This observation is interesting because both this attributs when we computed the correlation showed a correlation with the class "poisonous/edible" of nearly 0.

### Random Forest

The Random Forest is a model made up of many decision trees. Rather than just simply averaging the prediction of trees (which we could call a "forest"), this model uses two key concepts that gives it the name *random*:
+ Random sampling of training data points when building trees
+ Random subsets of features considered when splitting nodes

When training, each tree in a random forest learns from a random sample of the data points. The samples are drawn with replacement, known as *bootstrapping*, which means that some samples will be used multiple times in a single tree. The idea is that by training each tree on different samples, although each tree might have high variance with respect to a particular set of the training data, overall, the entire forest will have lower variance but not at the cost of increasing the bias. At test time, predictions are made by averaging the predictions of each decision tree. This procedure of training each individual learner on different bootstrapped subsets of the data and then averaging the predictions is known as bagging, short for bootstrap aggregating.

The other main concept in the random forest is that only a subset of all the features are considered for splitting each node in each decision tree. Generally this is set to `sqrt(n_features)` for classification meaning that if there are 16 features, at each node in each tree, only 4 random features will be considered for splitting the node. (The random forest can also be trained considering all the features at every node as is common in regression. These options can be controlled in the Scikit-Learn Random Forest implementation).

This approach improves the predictive accuracy and can seriously control and prevent overfitting.

The hyperparameters of a Random Forest (passed as arguments to RandomForestClassifier of sklear.ensemble) include the following ones:

+ n_estimators: the number of trees
+ criterion: the criterion which is responsible of the decision to split a node (default='gini')
+ max_depth: the maximum depth of each tree (default=None)
+ min_samples_split: the minimum number of samples in a node to be considered for further splitting (default=2)
+ max_features: the number of features considered for a split (default='sqrt')

In [42]:
%%time
clf_rf = RandomForestClassifier(random_state=SEED)
gs_rf = do_gridsearch_with_cv(clf_rf, RANDOM_FOREST_PARAMS, X_train, y_train, kf)

Wall time: 1min 30s


In [43]:
print_best_grid_search_scores_with_params(gs_rf)

Best grid scores on validation set:
1.000 (+/-0.000) for {'clf__criterion': 'entropy', 'clf__max_depth': 75, 'clf__max_features': 'sqrt', 'clf__n_estimators': 1000}
1.000 (+/-0.000) for {'clf__criterion': 'entropy', 'clf__max_depth': 75, 'clf__max_features': 'sqrt', 'clf__n_estimators': 500}
1.000 (+/-0.000) for {'clf__criterion': 'gini', 'clf__max_depth': 25, 'clf__max_features': 'sqrt', 'clf__n_estimators': 300}
1.000 (+/-0.000) for {'clf__criterion': 'gini', 'clf__max_depth': 25, 'clf__max_features': 'sqrt', 'clf__n_estimators': 500}
1.000 (+/-0.000) for {'clf__criterion': 'gini', 'clf__max_depth': 25, 'clf__max_features': 'sqrt', 'clf__n_estimators': 1000}
1.000 (+/-0.000) for {'clf__criterion': 'gini', 'clf__max_depth': 50, 'clf__max_features': 'sqrt', 'clf__n_estimators': 100}
1.000 (+/-0.000) for {'clf__criterion': 'gini', 'clf__max_depth': 50, 'clf__max_features': 'sqrt', 'clf__n_estimators': 300}
1.000 (+/-0.000) for {'clf__criterion': 'gini', 'clf__max_depth': 50, 'clf__max_f

In [44]:
gs_rf_score = gs_rf.score(X_test, y_test)
y_pred_rf = gs_rf.predict(X_test)
cm_rf = confusion_matrix(y_test, y_pred_rf)
cm_rf = cm_rf.astype('float') / cm_rf.sum(axis=1)[:, np.newaxis] # normalize the confusion matrix
cm_df = pd.DataFrame(cm_rf.round(3), index=["true edible", "true poisonous"], columns=["predicted edible", "predicted poisonous"])
cm_df

,predicted edible,predicted poisonous
true edible,1.0,0.0
true poisonous,0.0,1.0


Here the normalized confusion matrix is shown, as well as the result of the grid search on the top of it.

It shows that this model fits really well our data, achieving an overall F1-Score of 1.00 and not showing any bias towards any of the class.

In [45]:
plot_learning_curve(gs_dt.best_estimator_, "Learning Curve of the Random Forest", X_train, y_train, cv=5)

It can be seen from the graph that the test score of the Random Forest increases with the number of samples, reaching the maximum of 1.00. It shows that the Random Forest is not overfitting the train data, even if it reaches very early a score of 1.00 on the train set.

In the Random Forest Classifier the feature importances are obtained by computong it for all trees and taking the average.

In [46]:
feature_importance_rf = np.array(sorted(zip(X_train.columns, gs_rf.best_estimator_.named_steps['clf'].feature_importances_), key=lambda x: x[1], reverse=True))
plot_feature_importance(feature_importance_rf, "Feature importance in the Random Forest")

Interestingly enough, the most important features for the Random Forest are "odor", "gill-size" and "gill-color". The "spore-print-color" and the "stalk-shape" were not found as important as in the Decision Tree, while in this case we can understand why "gill-size" and "gill-color" are so important because also in the correlation computation thei showed respectively positive and negative correlation with the class "poisonous/edible".

### Support-Vector Machine

A linear Support-Vector Machine (SVM) finds the optimal hyperplane between the points of two classes such that the distance of the nearest points to the decision boundary is maximized. This distance is called margin.

If the data set is not linearly separable, we can map the samples $x$ into a feature space of higher dimensions: $x \rightarrow \phi(x)$ in which the classes can be linearly separed. This results in a non-linear decision boundary in the original dimensions.

As the vectors $x_i$ appear only in inner products in both the decision function and the learning law, the mapping $\phi(x)$ does not need to be explicitly specified. We can instead define a kernel function:
<br><center>$K(x_1,x_2) = \phi(x_1)^T\phi(X_2)$.</center>

In our gridsearch, two kernel functions have been used:
- linear kernel: $K(x_1,x_2)=x_1\cdot x_2$
- radial basis function: $K(x_1,x_2)=\exp(-\gamma(||x_1 - x_2||^2))$

The hyperparameters of a SVM (passed as arguments to SVC of sklearn.svm) include the following ones:

+ C: the inverse of the regularization strength (default=1.0)
+ kernel: the kernel used (default='rbf')
+ gamma: the higher the value, the classifier tries to exactl fit the training data set (default='auto-deprecated')

In [47]:
%%time
clf_svm = svm.SVC(random_state=SEED, probability=True)
gs_svm = do_gridsearch_with_cv(clf_svm, SVM_PARAMS, X_train, y_train, kf)

Wall time: 9min 45s


In [48]:
print_best_grid_search_scores_with_params(gs_svm, n=15)

Best grid scores on validation set:
1.000 (+/-0.000) for {'clf__C': 1, 'clf__gamma': 0.1, 'clf__kernel': 'rbf'}
1.000 (+/-0.000) for {'clf__C': 100, 'clf__gamma': 1, 'clf__kernel': 'rbf'}
1.000 (+/-0.000) for {'clf__C': 100, 'clf__gamma': 0.1, 'clf__kernel': 'rbf'}
1.000 (+/-0.000) for {'clf__C': 100, 'clf__gamma': 0.01, 'clf__kernel': 'rbf'}
1.000 (+/-0.000) for {'clf__C': 10, 'clf__gamma': 1, 'clf__kernel': 'rbf'}
1.000 (+/-0.000) for {'clf__C': 10, 'clf__gamma': 0.1, 'clf__kernel': 'rbf'}
1.000 (+/-0.000) for {'clf__C': 10, 'clf__gamma': 0.01, 'clf__kernel': 'rbf'}
1.000 (+/-0.000) for {'clf__C': 1, 'clf__gamma': 1, 'clf__kernel': 'rbf'}
0.998 (+/-0.004) for {'clf__C': 0.1, 'clf__gamma': 0.1, 'clf__kernel': 'rbf'}
0.986 (+/-0.001) for {'clf__C': 1, 'clf__gamma': 0.01, 'clf__kernel': 'rbf'}
0.984 (+/-0.012) for {'clf__C': 10, 'clf__kernel': 'linear'}
0.984 (+/-0.012) for {'clf__C': 1, 'clf__kernel': 'linear'}
0.976 (+/-0.012) for {'clf__C': 0.1, 'clf__gamma': 1, 'clf__kernel': 'rbf'}

In [49]:
gs_svm_score = gs_svm.score(X_test, y_test)
y_pred_svm = gs_svm.predict(X_test)
cm_svm = confusion_matrix(y_test, y_pred_svm)
cm_svm = cm_svm.astype('float') / cm_svm.sum(axis=1)[:, np.newaxis] # normalize the confusion matrix
cm_df = pd.DataFrame(cm_svm.round(3), index=["true edible", "true poisonous"], columns=["predicted edible", "predicted poisonous"])
cm_df

,predicted edible,predicted poisonous
true edible,1.0,0.0
true poisonous,0.0,1.0


Here the normalized confusion matrix is shown, as well as the result of the grid search on the top of it. From the grid we can see that the best results are obtained by the kernel 'rbf', so the problem is not linearly separable (the best score with a linear kernel is 0.984, that is high in general, but low considering the other results).

It shows that this model fits really well our data, achieving an overall F1-Score of 1.00 and not showing any bias towards any of the class.

In [50]:
plot_learning_curve(gs_svm.best_estimator_, "Learning Curve of SVM", X_train, y_train, cv=5)

It can be seen from the graph that the test score of the SVM increases with the number of samples, reaching the maximum of 1.00. It shows that the SVM is not overfitting the train data, even if it reaches very early a score of 1.00 on the train set.
   

### Results

In this section the goal is to compare the performance of every classification model tested previously. In order to do that, we consider several metrics such as precision, recall and f1-score that have already been defined. In addition to these, we add some more metrics that will help us to better compare the classifier. Let's remember the previous notation:
 + $TP$ = *True Positive* - number of samples where the prediction is positive and the true label is positive
 + $FP$ = *False Positive* - number of samples where the prediction is positive but the true label is negative
 + $TN$ = *True Negative* - number of samples where the prediction is negative and the true label is negative
 + $FN$ = *False Negative* - number of samples where the prediction is negative but the true label is positive

So, the **Accuracy** is defined as the percentage of the correct predictions
<br><center>$acc = \frac{TP+TN}{TP+FP+TN+FN}$</center>

Another metric defined is the **AUC**, or Area Under the Receiver Operating Characteristic curve, and to define this method, we need two more measures:
+ True Positive Rate ($TPR$) - the same as the recall: $TPR = recall = \frac{TP}{FN+TP}$
+ False Positive Rate ($FPR$) - proportion of negative data points that are wrongly classified as positive: $FPR = \frac{FP}{TN+FP}$

The **ROC** (Receiver Operating Characteristic) curve is drawn by coosing a number of different classification thresholds and compute $TPR$ and $FPR$. The area under this ROC curve (AUC) is computed to combine these two measures in one evaluation metric.

In [51]:
# code partially from https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html
def plot_roc_curve(classifiers, legend, title, X_test, y_test):
    trace1 = go.Scatter(
        x=[0, 1], 
        y=[0, 1], 
        showlegend=False,
        mode="lines",
        name="",
        line = dict(
            color = colors[0],
        ),
    )
    
    data = [trace1]
    aucs = []
    for clf, string, c in zip(classifiers, legend, colors[1:]):
        y_test_roc = np.array([([0, 1] if y else [1, 0]) for y in y_test])
        y_score = clf.predict_proba(X_test)
        
        # Compute ROC curve and ROC area for each class
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(2):
            fpr[i], tpr[i], _ = roc_curve(y_test_roc[:, i], y_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])

        # Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(y_test_roc.ravel(), y_score.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
        aucs.append(roc_auc['micro'])

        trace = go.Scatter(
            x=fpr['micro'], 
            y=tpr['micro'], 
            showlegend=True,
            mode="lines",
            name=string + " (area = %0.2f)" % roc_auc['micro'],
            hoverlabel = dict(
                namelength=30
            ),
            line = dict(
                color = c,
            ),
        )
        data.append(trace)

    layout = go.Layout(
        title=title,
        autosize=False,
        width=550,
        height=550,
        yaxis=dict(
            title='True Positive Rate',
        ),
        xaxis=dict(
            title="False Positive Rate",
        ),
        legend=dict(
            x=0.4,
            y=0.06,
        ),
    )
    fig = go.Figure(data=data, layout=layout)
    return aucs, iplot(fig, filename=title)

In the following figure the ROC curves for every classifier tested in the previous sections are shown on the same plot.

In [52]:
classifiers = [gs_lr, gs_lda, gs_knn, gs_dt, gs_rf, gs_svm]
classifier_names = ["Logistic Regression", "LDA", "KNN", "Decision Tree", "Random Forest", "SVM"]
auc_scores, roc_plot = plot_roc_curve(classifiers, classifier_names, "ROC curve", X_test, y_test)
roc_plot

As we can see, and as we could imagine reading the previous results, four out of the six tested classifier performed really well achieving a AUC value of 1.0.

The worst performance is done by the LDA classifier, and also the Logistic Regression one is fairly below the other four.

We compute now all the metrics introduced in this work for every model, and put them in a table to have a better idea of the performance of every classifier.

In [53]:
accs = []
recalls = []
precision = []
results_table = pd.DataFrame(columns=["accuracy", "precision", "recall", "f1", "auc"])
for (i, clf), name, auc in zip(enumerate(classifiers), classifier_names, auc_scores):
    y_pred = clf.predict(X_test)
    row = []
    row.append(accuracy_score(y_test, y_pred))
    row.append(precision_score(y_test, y_pred))
    row.append(recall_score(y_test, y_pred))
    row.append(f1_score(y_test, y_pred))
    row.append(auc)
    row = ["%.3f" % r for r in row]
    results_table.loc[name] = row
    
results_table

,accuracy,precision,recall,f1,auc
Logistic Regression,0.967,0.959,0.974,0.966,0.990
LDA,0.948,0.951,0.941,0.946,0.957
KNN,1.000,1.000,1.000,1.000,1.000
Decision Tree,1.000,1.000,1.000,1.000,1.000
Random Forest,1.000,1.000,1.000,1.000,1.000
SVM,1.000,1.000,1.000,1.000,1.000


This table shows how the last four classifier are perfect in this classification problem, and between Logistic Regression and LDA, LDA has a worst recall that in this particular analysis make it a worse classifier (we don't want that a poisonous mushroom is classified as edible). It also shows a lower f1-score as we noticed in the LDA section, due to a bias on the slightly majority class of "edible"

# Conclusions

The objective of this work was to analyze this mushroom dataset, trying to understand relations between variables and other information, with the goal of distinguishing the poisonous mushroom from the edible ones based on the other attributes.

The PCA transformation performed in the pre-processing part was surprisingly useful, and in case of need of dimensionality reduction it is strongly suggested to reduce the dataset to 9 PCs or even 5 PCs as shown in the results.

In the classification section, after the testing of the six different classifier, we may assume that this dataset is a very simple one, with well-separed classes and with no outliers (we can say that because it's very uncommon to achieve an accuracy of 1.00!!).